In [2]:
%load_ext autoreload


In [3]:
%autoreload 1


In [4]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np
import json
import pickle
from transformers import StoppingCriteria, StoppingCriteriaList
import re
%aimport vicuna_setup
from vicuna_pubmedqa_eval import model_pubmed_evaluator
import pandas as pd
import csv
from collections import Counter
from evaluation import pub_scores

In [5]:
path="/home/vicuna-weights-7B"
model_path="/home/vicuna-weights-7B"
tokenizer = LlamaTokenizer.from_pretrained(path)

In [6]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(model_path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.52s/it]


In [7]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

pubmed_config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=3,
        min_length=1,
        max_time=3,
        max_new_tokens=3, # 3
        forced_bos_token_id=[tokenizer.encode("yes")[1], tokenizer.encode("no")[1], tokenizer.encode("maybe")[1]],
        
    )

In [8]:
#ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=0)

In [ ]:
len(ans_dict)

In [ ]:
with open("/home/medical-llama/pubmed-pred/base.json", "w") as json_file:
    json.dump(ans_dict, json_file)

In [ ]:
pub_scores("/home/medical-llama/pubmed-pred/base.json")

In [ ]:
ans_dict

In [ ]:
acc,f1_macro=[],[]
for i in range(5):
    ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=i)
    with open("/home/medical-llama/pubmed-pred/base.json", "w") as json_file:
        json.dump(ans_dict, json_file)
    accuracy,f1=pub_scores("/home/medical-llama/pubmed-pred/base.json")
    acc.append(accuracy)
    f1_macro.append(f1)


In [ ]:
print(acc)
print(f1_macro)

In [ ]:
print("Accuracy: ",np.mean(acc))
print("F1 Macro: ",np.mean(f1_macro))

In [5]:
test_data = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")


In [8]:
config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=128,
        min_length=1,
        max_time=10,
        max_new_tokens=128,)

In [9]:
print(vicuna_inference("can i take paracetamol & drink alcohol ?",model=model,tokenizer=tokenizer,config=config))


It is generally not recommended to mix paracetamol with alcohol as it
can cause stomach problems and potentially lead to liver damage. If
you have any concerns or questions about taking paracetamol, it's best
to consult with a healthcare professional. 


In [18]:
pubmed_config=GenerationConfig(
        temperature=0.4,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=3,
        min_length=1,
        max_time=3,
        max_new_tokens=3, # 3
        forced_bos_token_id=[tokenizer.encode("yes")[1], tokenizer.encode("no")[1], tokenizer.encode("maybe")[1]],
        
    )

In [12]:
df = pd.read_csv("/home/medical-llama/Pubmedqa/pubmedqa/data/pubmedqa_test.csv")

In [13]:
def pubmed_template(Question, Context,id=0):

    prompt_template=f"""
    
    your task is to answer research questions relating to medical domain using yes/no/maybe responses. Answer as yes if the context supports the question , answer as no if the context does not support the question , answer as maybe if the context is not clear enough to answer the question.      

    ###
            
    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are dogs trainable?

    Answer: yes

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Can dogs be trained to detect and alert individuals to the presence of specific allergens?

    Answer: maybe

    ###

    Context: Research involving dogs has been instrumental in advancing our understanding of various scientific fields. Dogs are highly trainable and sociable animals.

    Question: Are Dogs highly introverted & reserved ?

    Answer: no 

    ###

    Context:
    {Context}

    Question:
    {Question}

    Answer: 
    """
    return prompt_template

In [14]:
vicuna_setup.vicuna_inference(pubmed_template(Question=str(df['Question'][5]),Context=str(df['Context'][5])),model=model,tokenizer=tokenizer,config=pubmed_config)


tokenzied input
tensor([[    1, 29871,    13, 21140,   340,   338,   385, 15278,   393, 16612,
           263,  3414, 29889, 14350,   263,  2933,   393,  7128,  2486,  1614,
          2167,   278,  2009, 29889,    13, 29871,    13,  2277, 29937,  2799,
          4080, 29901,    13,    13,   268,    13,  1678,   596,  3414,   338,
           304,  1234,  5925,  5155,  1104,  1218,   304, 16083,  5354,   773,
          4874, 29914,  1217, 29914, 26026, 20890, 29889,   673,   408,  4874,
           565,   278,  3030, 11286,   278,  1139,  1919,  1234,   408,   694,
           565,   278,  3030,   947,   451,  2304,   278,  1139,  1919,  1234,
           408,  5505,   565,   278,  3030,   338,   451,  2821,  3307,   304,
          1234,   278,  1139, 29889,   539,    13,    13,  1678,   835,    13,
           632,    13,  1678, 15228, 29901, 10550, 21677, 26361,   756,  1063,
         11395,   284,   297,  3061, 19985,  1749,  8004,   310,  5164, 16021,
          4235, 29889,   360, 12099,

'Yes, spinal subdural hemat'

In [20]:
ans_dict={}
for i in range(int(df.shape[0])):
    pmid=str(df['ID'][i])
    ans=vicuna_setup.vicuna_inference(pubmed_template(Question=df['Question'][i],Context=df['Context'][i]),model=model,tokenizer=tokenizer,config=pubmed_config)
    ans_dict[pmid]=ans

In [22]:
ans_dict

{'12377809': 'Yes,',
 '26163474': 'Yes,',
 '19100463': 'Yes,',
 '18537964': 'Yes,',
 '12913878': 'Yes,',
 '12765819': 'Yes,',
 '25475395': 'Yes,',
 '19130332': 'Yes,',
 '9427037': 'Yes,',
 '24481006': 'Yes,',
 '8165771': 'Yes,',
 '22680064': 'Yes,',
 '22540518': 'Yes.',
 '20629769': 'Yes,',
 '21726930': 'Yes,',
 '21481154': 'Yes,',
 '22902073': 'Yes,',
 '26370095': 'Yes,',
 '18041059': 'Yes,',
 '15041506': 'Yes,',
 '11146778': 'Yes,',
 '27281318': 'Yes,',
 '21645374': 'Yes.',
 '9465206': 'Yes,',
 '25887165': 'Yes,',
 '15995461': 'Yes,',
 '21850494': 'Yes,',
 '19106867': 'Yes,',
 '21342862': 'Yes,',
 '24352924': 'Yes,',
 '16147837': 'Answer:',
 '26879871': 'Yes,',
 '15918864': 'Answer:',
 '22075911': 'Yes,',
 '11035130': 'Yes,',
 '21228436': 'Yes,',
 '11833948': 'Yes,',
 '17682349': 'Yes,',
 '17355582': 'Yes,',
 '15597845': 'Yes,',
 '10158597': 'Yes.',
 '27549226': 'Yes,',
 '26348845': 'Yes.',
 '25588461': 'Yes,',
 '23359100': 'Yes,',
 '26548832': 'Answer:',
 '25756710': 'Yes,',
 '20297

In [23]:
element_counts = Counter(list(ans_dict.values()))

for element, count in element_counts.items():
    print(f"{element}: {count}")

Yes,: 401
Yes.: 27
Answer:: 43
Maybe.: 2
No,: 19
Answer: Maybe: 1
Maybe,: 3
Yes: 1
The provided: 1
The answer: 1
Sternal: 1
